
# ECE795 Advanced Big Data Analytics Homework 4 (80 Points + 20 Bonus Points) Due Date: March 8, 2022 (by 1:00pm)

Set up Pyspark Environment.

Tips for Colab:

1. You will be disconnected if you are idle for more than 90 minutes and will be mandatorily disconnected after 12 hour connection. 

2. Once you got disconnected, you need to execute the codes from the beginning to setup the environment again.

3. For the purpose of homework, it should be sufficient since each problem should not take more than 5 minutes to generate the results.

4. To facilitate the use of Colab, you can use "MainMenu - Runtime - Run all” to run all the cells in the notebook. So you do not have to click each cell to setup the environment.

In [ ]:
!pip install pyspark
!pip install -q findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Congrats! Your Colab is ready to run Pyspark.

# Read input text file to RDD 

Download the input data from [here](https://raw.githubusercontent.com/umddm/ECE795_Homeworks/master/clust_B.txt).

In [ ]:
!wget https://raw.githubusercontent.com/umddm/ECE795_Homeworks_Spring2022/homework_4/StudentsPerformance.csv

Now that we have input data, we can start to do the homework. 

## Question 1: Please rewrite the given function only with the limited operation. (15 Points)

### Expected output:
```
.sum() by .map() and .reduce().

can be rewritten as .reduce(lambda a, b: a + b)
```

In [ ]:
#sc.textFile
from pyspark import SparkConf, SparkContext
sc = SparkContext.getOrCreate()

#Sample
Sample = sc.textFile('StudentsPerformance.csv')
Sample = Sample.map(lambda x: x.replace('"', ''))
header = Sample.first()
Sample = Sample.filter(lambda line: line != header)
Sample.map(lambda x: int(x.split(',')[-1])).sum() == Sample.map(lambda x: int(x.split(',')[-1])).reduce(lambda a, b: a + b) 
#Output is Ture



### Question 1. 1: (5 Points)
Please implement .count() by .map() and .reduce().

### Question 1. 2: (5 Points)
Please implement .stdev() by .map() and .reduce().

Round to the second decimal places

### Question 1. 3: (5 Points)
Please implement .reduceByKey() by partitionBy(), map() and glom()

A hint to solve this problem is that a custom partitioner should be implemented and use glom to group the data.


## Question 2: Please create an RDD, which contains range(12) with 3 partitions. Then count the minimum of the partition 1, the maximum of the partition 2, and the sum of the third partition. (15 Points)


### Sample:
```
min of the partition 1 = 1
max of the partition 2 = 10
sum of the partition 3 = 20
```

In [ ]:
#Question_2

#Fill out here

## Question 3: Why will pre-partition benefit spark job? Please write a short explanation and a code to prove your answer. (20 Points)

### Hint:
`ToDebugString()` can be used to debug. Detailed description can be found at https://spark.apache.org/docs/latest/api/java/org/apache/spark/rdd/RDD.html#toDebugString--

You can use `toDebugString()` like below:
```
rdd = sc.textFile('clust_B_part1.txt').toDebugString() 
```

In [ ]:
#Question_3

#Fill out here

## Question 4: Please implement the K-Means algorithm described in the lecture slide. And test the implemented algorithm with data in "s1.txt" (k=15). (30 Points)


The coordinates of the center change less than 0.1 are considered as unchanged.

### Input data format:
Each row describe a point using two integers, which are its x and y coordinates.
The ID of the points is the row number, i.e., the point in the first row has ID=1, the point in the second row has ID=2, etc.

### Expected output format:
Each line describes a cluster including the cluster ID and the coordinate of its center
```
Cluster 1 (12.00, 67.33)
Cluster 2 (33.25, 23.02)
...
Cluster 15 (123.43, 456.30)
```

### Hint:
`zipWithIndex()` can be used to add index to the rdd. Detailed description can be found at https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.zipWithIndex

In [ ]:
# Please download the data using the following commands
!wget http://cs.joensuu.fi/sipu/datasets/s1.txt

In [ ]:
#Question_4

#Fill out here

## Question 5: Please implement the One-Dimension Gaussian Mixture model described in the following. And test the implemented algorithm with the **first column** of data in "s1.txt" (k = 15). (Bonus: 20 Points)


No matter whether the model converges, run the model for 10 iterations

### Algorithm Description

This algorithm is similar to K-Means, clustering the given data. Different from K-means, Gaussain Mixture model defines each cluster by mean, variance and weight, instead of the coordination of the cluster center

- Step1: Assign initial means, variance, and weights of each cluster

  - Each cluster is defined by three value $C(\mu, \sigma^2, w)$. 
  - Mean $\mu$ can be initialized by the value of a randomly selected data. 
  - Variance $\sigma^2$ can be initialized by the variance of 15 randomly selected data. 
  - Weight $w$ can be initialized by 1.

- Step2: Go to an iteration

  - Compute a score for each data point $x$ and all the clusters, using the provided *pdf* function.
  - Assign each data point to the cluster with the highest score
  - Compute normalized score of each data point by: $S_{i,k}=\dfrac{s_{i,k}}{\sum_{j=1}^{K}{s_{i,j}}}$ ($K$ is the number of clusters)
  - For each cluster k 
    - Update mean of the cluster by: $\mu_k'=\dfrac{\sum_{i=1}^{N}(x_iS_{i,k})}{0.001+\sum_{i=1}^{N}{S_{i,k}}}$
    - Update variance of the cluster by: $\sigma_k'^2 = \dfrac{\sum_{i=1}^{N}[S_{i,k}(x_i - \mu)^2]}{0.001+\sum_{i=1}^{N}{S_{i,k}}}$
    - Update weight of the cluster by: $\dfrac{\sum_{i=1}^{N}S_{i,k}}{N}$
    - $N$ is the number of data points in the dataset
    - $s_{i,k}$ is the score of the $i$-th data point for the cluster k

### Expected output format:
Each line describes a cluster including the cluster ID, mean, variance, and weight.
```
Cluster 1: mean=12.00, variance=1.23, weight=2.23
Cluster 2: mean=5.23, variance=2.33, weight=4.23
...
Cluster 15: mean=17.20, variance=0.64, weight=1.68
```

In [ ]:
def pdf(data: float, mean: float, variance: float, weight: float):
  s1 = 1/(np.sqrt(2*np.pi*variance))
  s2 = np.exp(-(np.square(data - mean)/(2*variance)))
  return s1 * s2 * weight

In [ ]:
#Question_5

#Fill out here